In [11]:
from langchain.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain.document_loaders import WebBaseLoader
from langchain.utilities import SerpAPIWrapper

class ResearchAITools:
    def __init__(self):
        self.wikipedia_tool = WikipediaQueryRun()
        self.duckduckgo_tool = DuckDuckGoSearchRun()
    
    def search_wikipedia(self, query: str):
        """위키피디아에서 검색하여 결과 반환"""
        return self.wikipedia_tool.run(query)
    
    def search_duckduckgo(self, query: str, num_results: int = 5):
        """덕덕고에서 검색하여 결과 반환"""
        return self.duckduckgo_tool.run(query, num_results=num_results)
    
    def scrape_website(self, url: str):
        """주어진 URL의 콘텐츠를 가져옴"""
        loader = WebBaseLoader(url)
        documents = loader.load()
        return "\n".join([doc.page_content for doc in documents])
    
    def save_to_file(self, filename: str, content: str):
        """리서치 결과를 텍스트 파일로 저장"""
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(content)
        print(f"Results saved to {filename}")

# 사용 예시
tools = ResearchAITools()
query = "Artificial Intelligence"

# 위키피디아 검색
data_wiki = tools.search_wikipedia(query)
print("Wikipedia Result:\n", data_wiki)

# 덕덕고 검색
data_ddg = tools.search_duckduckgo(query)
print("DuckDuckGo Results:\n", data_ddg)

# 첫 번째 검색 결과 URL 크롤링
if data_ddg:
    scraped_content = tools.scrape_website(data_ddg[0])
    print("Scraped Content:\n", scraped_content[:500])  # 앞부분만 출력
    
    # 결과 저장
tools.save_to_file("research_results.txt", f"Wikipedia:\n{data_wiki}\n\nDuckDuckGo:\n{data_ddg}\n\nScraped Content:\n{scraped_content}")


ValidationError: 1 validation error for WikipediaQueryRun
api_wrapper
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing